In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import proplot as pplt
from scipy.fftpack import * 

# 进行年际变率的计算、绘制

2022.02.17

年际变率的计算和绘制，将T2M 和 PRECIP 放置在一起

添加空间平均态的统计量表格

## 读入数据

### 日降水

In [ ]:
dir_in = "/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA_large/pre/ordata/"
filename_obs  = "obsmerge_pre_98-17.nc"
filename_vr     = "vr_pre_98-17.nc"
filename_rcm    = "rcm_pre_98-17.nc"

ds_or = {}
ds_or['obs'] = xr.open_dataset(dir_in + filename_obs)
ds_or['vr']     = xr.open_dataset(dir_in + filename_vr)
ds_or['rcm']    = xr.open_dataset(dir_in + filename_rcm  )
# 提取变量
var = {}
var['precip'] = {}
var['precip']['obs'] = ds_or['obs']['premerge']#[:,  :, :]
var['precip']['vr'] = ds_or['vr']['precip_MPAS']
var['precip']['rcm'] = ds_or['rcm']['precip_MPAS']

var['precip']['obs'] = var['precip']['obs'].reset_coords(names = 'lev', drop = True) # 去除掉obs中多余的lev coords

# change coords
var_list = ['obs', 'vr', 'rcm']
for i in var_list:
    rename_dict = dict(zip(var['precip'][i].coords.keys(), var['precip']['obs'].coords.keys()))
#     # show converting coords
    for rename_i in rename_dict:
        print(rename_i + " -----converting to----- " + rename_dict[rename_i])

    var['precip'][i] = var['precip'][i].rename(rename_dict)
    var['precip'][i]._coords = var['precip']['obs']._coords
    var['precip'][i] = var['precip'][i].rename(i)


### 日平均气温

In [ ]:
dir_in = "/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA_large/t2m/ordata/"
filename_obs  = "mask_sel_CN05.1_Tm_1961_2018_daily_025x025.nc"
filename_vr     = "mask_mean_t2m_98-17_VR.nc"
filename_rcm    = "mask_mean_t2m_98-17_RCM.nc"

ds_or = {}
ds_or['obs'] = xr.open_dataset(dir_in + filename_obs)
ds_or['vr']     = xr.open_dataset(dir_in + filename_vr)
ds_or['rcm']    = xr.open_dataset(dir_in + filename_rcm  )
# 提取变量
var['t2m'] = {}
var['t2m']['obs'] = ds_or['obs']['tm']#[:,  :, :]
var['t2m']['vr'] = ds_or['vr']['t2m'] - 273.15
var['t2m']['rcm'] = ds_or['rcm']['t2m'] - 273.15

# var['t2m']['obs'] = var['t2m']['obs'].reset_coords(names = 'lev', drop = True) # 去除掉obs中多余的lev coords

# change coords
var_list = ['obs', 'vr', 'rcm']
for i in var_list:
    rename_dict = dict(zip(var['t2m'][i].coords.keys(), var['t2m']['obs'].coords.keys()))
#     # show converting coords
    for rename_i in rename_dict:
        print(rename_i + " -----converting to----- " + rename_dict[rename_i])

    var['t2m'][i] = var['t2m'][i].rename(rename_dict)
    var['t2m'][i]._coords = var['t2m']['obs']._coords
    var['t2m'][i] = var['t2m'][i].rename(i)


## 计算部分

### 计算年际序列

In [ ]:
var_selmonth = {}
var_interannual = {}
time_for_groupby = {}

for vartype in ['precip','t2m']:
    var_selmonth[vartype] = {}
    time_idx_mjja = var[vartype]['obs'].time.dt.month.isin([5, 6, 7, 8])
    for mod_name in ['obs', 'vr', 'rcm']:
        var_selmonth[vartype][mod_name]  = var[vartype][mod_name].isel(time = time_idx_mjja)

    # 获取年际时间序列 var_interannual{} dict
    time_for_groupbyyears = var_selmonth[vartype]['vr'].time.dt.year
    var_interannual[vartype] = {}
    for mod_name in ['obs', 'vr', 'rcm']:
        var_interannual[vartype][mod_name] = var_selmonth[vartype][mod_name].\
            groupby(time_for_groupbyyears).mean(dim = 'time')

### 计算年际标准差

In [ ]:
std_interannual = {}

for vartype in ['precip','t2m']:
    std_interannual[vartype] = {}
    for mod_name in ['obs', 'vr','rcm']:
        std_temp = var_interannual[vartype][mod_name].std(dim = "year")
        coords2d = {"lon":var_interannual[vartype][mod_name].lon, "lat":var_interannual[vartype][mod_name].lat}
        std_interannual[vartype][mod_name] = xr.DataArray(std_temp, coords = coords2d, name = 'variance')

### 计算年际变率的相关系数、均方根误差

In [ ]:
import skill_metrics as sm
std_1d = {}
for vartype in['precip','t2m']:
    std_1d[vartype] = {}
    for mod_name in ['obs', 'vr', 'rcm']:
        temp_obs = std_interannual[vartype]['obs'].values
        temp_mod = std_interannual[vartype][mod_name].values

            # 获取1D的平均态 并以RCM为基础进行NAN的MASK
        index_not_nan = ( ~np.isnan(std_interannual[vartype]['obs'])) &\
            (~np.isnan(std_interannual[vartype]['vr'])) &\
            (~np.isnan(std_interannual[vartype]['rcm']) )
        var_1d_temp = xr.where(index_not_nan, var_interannual[vartype][mod_name], np.nan).values.ravel()
        std_1d[vartype][mod_name] = var_1d_temp[~np.isnan(var_1d_temp)]


metrics_std = {}
for vartype in['precip','t2m']:
    metrics_std[vartype] = {}
    for mod_name in ['vr','rcm']:
        metrics_std[vartype][mod_name] = pd.DataFrame(index = [mod_name], columns=['Bias','RMSE','Corr.']) 
        temp_obs = std_1d[vartype]['obs']
        temp_mod = std_1d[vartype][mod_name]

        metrics_std[vartype][mod_name].loc[mod_name]['Bias'] = sm.bias(temp_mod, temp_obs)
        metrics_std[vartype][mod_name].loc[mod_name]['RMSE'] = sm.rmsd(temp_mod, temp_obs)
        metrics_std[vartype][mod_name].loc[mod_name]['Corr.'] = np.corrcoef(temp_mod, temp_obs)[0,1]

        # # 对style进行调整，用于后续展示表格
        # # https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html
    # metrics_std[vartype] = metrics_std[vartype].style.format("{:,.3f}")

## 绘图部分

In [ ]:
# 国内政区图的绘制
# Load the border data, CN-border-La.dat is download from
# https://gmt-china.org/data/CN-border-La.dat
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.patches as mpatches

from matplotlib import pyplot as plt
import proplot as plot
import cmaps
from matplotlib.font_manager import FontProperties

In [ ]:
def draw_borders(axs):
    # for ax_ind in axs:
    # for line in borders:
    # axs.plot(line[0::100], line[1::100], lw = 0.5, color='gray',transform=ccrs.Geodetic())
    # axs.plot(line[0::10], line[1::10], lw = 0.4, color='black',transform=ccrs.Geodetic())
    ##---- 使用shp文件添加
    ## shapefile数据下载的位置：
    ## http://gaohr.win/site/blogs/2017/2017-04-18-GIS-basic-data-of-China.html
    # world_border_shapefile = "/m2data2/yycheng/data_stage/CN-border/World/country.shp"
    river_border_shapefile =  "/raid52/yycheng/MPAS/REFERENCE/MODEL_CONSTANT/R1/" + "hyd1_4l.shp"
    southsea_shapefile     = "/m2data2/yycheng/data_stage/CN-border/SouthSea/" + "southsea_island.shp"
    ninelines_shapefile     = "/m2data2/yycheng/data_stage/CN-border/SouthSea/" + "nine_lines.shp"
    ## 来源： 沛沛的诸省 + 诸岛
    bou24p_shapefile     = "/m2data2/yycheng/data_stage/CN-border/peipeihelp/" + "bou2_4p.shp"
    ## 来源： https://www.resdc.cn/data.aspx?DATAID=200
    province_shapefile     = "/m2data2/yycheng/data_stage/CN-border/CN-sheng/" + "change_proj_CN-sheng-A.shp"

    for ax in axs:
        # world     = shpreader.Reader(world_border_shapefile).geometries()
        river     = shpreader.Reader(river_border_shapefile).geometries()
        # bou24p    = shpreader.Reader(bou24p_shapefile).geometries()
        ninelines = shpreader.Reader(ninelines_shapefile).geometries()
        province  = shpreader.Reader(province_shapefile).geometries()
        ax.add_geometries(river, ccrs.PlateCarree(), facecolor='none', edgecolor='b', linewidth=0.4, zorder=1)
        # ax.add_geometries(world, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.4, zorder=1)
        # ax.add_geometries(bou24p, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1) # 沛沛map
        ax.add_geometries(province, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1) # 地资所
        ax.add_geometries(ninelines, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1)

In [ ]:
# import proplot as plot
from matplotlib import pyplot as plt
import proplot as plot
import collections

#----- create plot -----
fig, axs = plot.subplots(ncols = 3, nrows = 2, share = 0, proj=('cyl'))
m_contour_list = [] # 用于保存contour设置，后续设置colorbar使用

#----- 添加海洋以及行政区划 -----
draw_borders(axs)

#----- colorbar ticks 统一设置 -----
std_ticks = {}
std_ticks_diff = {}
# std_ticks['precip'] = np.concatenate( (np.linspace(0,5,21),np.linspace(6,8,3)) , axis = 0)
std_ticks['precip'] = np.linspace(0,3,11)
std_ticks_diff['precip'] = np.concatenate( (np.linspace(-1,-0.2,5),np.linspace(0,3.,6)), axis = 0 )
std_ticks['t2m'] = np.arange(0.2, 0.75, 0.05)
std_ticks_diff['t2m'] = np.concatenate( (np.linspace(-0.3,-0.1,3), np.linspace(0,0.5,6)), axis = 0)


# plot contourf and titile axs

def nested_dict(): # 用于进行绘制的多重dict的设置
    return collections.defaultdict(nested_dict)
mcontourf_dict = nested_dict()

# STD
plot_ind = 0

for vartype_ind, vartype in enumerate(['t2m','precip']):
    for mod_ind, mod_name in enumerate(['obs','vr','rcm']):
    
        cmap_std = cmaps.WhiteBlueGreenYellowRed
        # cmap_std_diff = cmaps.ncl_default
        # cmap_std_diff = cmaps.NCV_jaisnd
        cmap_std_diff = cmaps.NCV_blue_red
        lon = std_interannual[vartype]['vr'].lon.values
        lat = std_interannual[vartype]['vr'].lat.values
        midnorm = pplt.DivergingNorm(vcenter = 0, fair = True)

        # std
        # if (vartype == 't2m'):
            # axs_small_temp = add_southchinasea([axs[plot_ind]]) # 生成南海小图的axs
        if (mod_name == 'obs'):
            mcontourf_dict[vartype][mod_name] = axs[plot_ind].contourf(lon, lat, std_interannual[vartype][mod_name].values,\
            levels=std_ticks[vartype],cmap=cmap_std, norm = 'segmented', extend = 'both')
            # if (vartype == 't2m'):
                # axs_small_temp.contourf(lon, lat, std_interannual[vartype][mod_name].values,\
                # levels=std_ticks[vartype],cmap=cmap_std, norm = 'segmented', extend = 'both')
        else:
            mcontourf_dict[vartype][mod_name] = axs[plot_ind].contourf(lon, lat, std_interannual[vartype][mod_name].values - std_interannual[vartype]['obs'].values,\
            levels=std_ticks_diff[vartype], cmap=cmap_std_diff, norm = midnorm, discrete = True, extend = 'both')
            # if (vartype == 't2m'):
            #     axs_small_temp.contourf(lon, lat, std_interannual[vartype][mod_name].values - std_interannual[vartype]['obs'].values,\
            #     levels=std_ticks_diff[vartype], cmap=cmap_std_diff, norm = midnorm, discrete = True, extend = 'both')

        # axs[plot_ind].format(title = mod_name.upper())
        plot_ind = plot_ind + 1

#----- add color bar-----
units = {}
units['precip'] = "[mm/d]"
units['t2m']    = r"$[^\circ C]$"

plot_ind = 2

for vartype in ['t2m', 'precip']:
    # obs colorbar
    axs[plot_ind].colorbar(mcontourf_dict[vartype]['obs'], loc='r', width=0.1,rows = (2,2),norm = 'segmented',
    ticklabelsize=5,ticks=std_ticks[vartype], title='standard deviation ' + units[vartype], extend = 'both')
    # bias colorbar
    axs[plot_ind].colorbar(mcontourf_dict[vartype]['vr'], loc='r', width=0.1,rows = (2,2),
    ticklabelsize=5,ticks=std_ticks_diff[vartype], title="std differences " + units[vartype], \
    norm = midnorm, extend = 'both')
    plot_ind = plot_ind + 3

# ----- format setting -----
axs.format(
abc=True,
abcloc = 'ul',
#----- 地图底图设置 -----
# reso = 'x-hi',
reso = 'med',
# coast = False,
coast = True,
coastlinewidth = 0.4,
borders = False,
lakes = False,
land  = False,
ocean = False,
# cartopyautoextent = True, 
# borderslinewidth=.5,
#-----GEO axis-----
labels = False,
longrid  = True,
latgrid  = True,
lonlim=(70, 140), latlim=(5, 60),
gridlabelsize = 5,
gridminor = True,
lonlocator = np.arange(70,142,10),
latlocator = np.arange(5,65+2,10),
lonminorlocator = np.arange(70,140+2,2),
latminorlocator = np.arange(5,65+2,2),
#-----line label-----
suptitle="interannual STD",
titleweight = 'bold',
toplabels=('OBS', 'VR', 'RCM'),
leftlabels=('T2m','daily precipitation'),
)
# axs[0:3].format(lonlim=(70, 140), latlim=(5, 60),)

# 合并子图之后控制边界的labels绘制
axs[:-1,0].format(labels = True, lonlabels = False, latlabels = True)
axs[-1,1:].format(labels = True, lonlabels = True, latlabels = False)
axs[-1,0].format(labels = True, lonlabels = True, latlabels = True)

# ----- add table on axis -----
# 参考回答：https://stackoverflow.com/questions/54150557/how-to-show-dataframe-index-name-on-a-matplotlib-table
# 使用bbox调整表格的大小
plot_ind = 0
for vartype in ['t2m','precip']:
    for mod_name in ['obs','vr', 'rcm']:
        if (mod_name == 'obs'):
            plot_ind +=1
            continue

        plot_table = metrics_std[vartype][mod_name].applymap("{:,.2f}".format)
        table = axs[plot_ind].table(cellText=np.matrix.round(plot_table.values.astype('float'),3),\
                    colLabels=plot_table.columns, loc = 'center',colColours=['gainsboro'] * 3,\
                    colWidths = [0.13]*3, bbox = [0.09, .82, 0.35, 0.15], zorder = 10)
        # table.set_fontweight('roman')
                    #  colColours=['gainsboro'] * len(plot_table), colLabels=plot_table.columns, loc='center',
                    #  colWidths= [0.12]*(len(plot_table.columns)))
        for (row, col), cell in table.get_celld().items():
            if ((row == 0)):
                cell.set_text_props(fontproperties=FontProperties(weight='bold'))
        plot_ind +=1

#----- save figure -----
fig.patch.set_facecolor('white')
fig.savefig('./output_pic/t2m_InterannualStd_2022.02.19.png', dpi=600, facecolor= "white")